<a href="https://colab.research.google.com/github/Adhiksha007/AI-Bootcamp/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/book-recommendation-dataset


In [3]:
data = pd.read_csv(path + "/Ratings.csv")

In [5]:
data = data[:10000]

In [6]:
data

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
9995,243,0425164403,0
9996,243,0440224764,0
9997,243,0440225701,0
9998,243,0440226430,0


In [9]:
data.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [11]:
user_book_matrix = data.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)

In [12]:
user_book_matrix

ISBN,0002005018,0002231115,0002232766,0002240114,000225669X,000254794,0002558122,0002740230,0006128831,0006379702,...,9782264014184,9782922145441,9788401499883,9871138016,9995585227,B0000BLD7X,B158991965,DITISEENSOORT,N3453124715,O6712345670
User-ID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
user_similarity = cosine_similarity(user_book_matrix)

In [14]:
user_similarity_df = pd.DataFrame(user_similarity, index=user_book_matrix.index, columns=user_book_matrix.index)

In [17]:
def recommendation_books(user_id, similarity_matrix, user_book_matrix, n=5):
  if user_id not in user_book_matrix.index:
    print("User Not Found in dataset")
    return []

  similar_users = similarity_matrix[user_id].sort_values(ascending=False).drop(user_id)
  recommended_books = {}
  for user, similarity in similar_users.items():
    books = user_book_matrix.loc[user]
    for book, rating in books[books > 0].items():
      if book not in user_book_matrix.loc[user_id] or user_book_matrix.loc[user_id, book] == 0:
        recommended_books[book] = recommended_books.get(book, 0) + rating * similarity

  recommended_books = sorted(recommended_books.items(), key=lambda x: x[1], reverse=True)[:n]
  return [book for book, scores in recommended_books]

In [18]:
recommendation_books(276729, user_similarity_df, user_book_matrix)

['0061092800', '0525942998', '0553564528', '0316569321', '0345435168']